In [8]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('连接成功，下一个有效订单ID：', self.nextorderId)

def run_loop():
    app.run()

# 创建应用实例
app = IBapi()

# 连接到TWS (127.0.0.1, 端口7497为纸交易，7496为实盘，客户端ID任意)
app.connect('127.0.0.1', 7496, 123)

# 启动API线程
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# 等待连接确认
while not hasattr(app, 'nextorderId'):
    print('等待连接...')
    time.sleep(1)

print('已连接到IBKR')
time.sleep(5)
app.disconnect()


ERROR -1 2104 市场数据场连接正常:usfarm.nj
ERROR -1 2104 市场数据场连接正常:hfarm
ERROR -1 2104 市场数据场连接正常:usfuture
ERROR -1 2104 市场数据场连接正常:cashfarm
ERROR -1 2104 市场数据场连接正常:usopt
ERROR -1 2104 市场数据场连接正常:eufarmnj
ERROR -1 2104 市场数据场连接正常:usfarm
ERROR -1 2106 历史市场数据场连接正常:euhmds
ERROR -1 2106 历史市场数据场连接正常:apachmds
ERROR -1 2106 历史市场数据场连接正常:fundfarm
ERROR -1 2106 历史市场数据场连接正常:ushmds
ERROR -1 2158 Sec-def数据场连接正常:secdefsg


等待连接...
连接成功，下一个有效订单ID： 1
已连接到IBKR


In [10]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time

class MarketDataApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.market_data = {}
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('连接成功')
        
    def tickPrice(self, reqId, tickType, price, attrib):
        """接收价格数据"""
        if tickType == 1:  # 买价
            self.market_data[f'{reqId}_bid'] = price
        elif tickType == 2:  # 卖价
            self.market_data[f'{reqId}_ask'] = price
        elif tickType == 4:  # 最新价
            self.market_data[f'{reqId}_last'] = price
        elif tickType == 6:  # 最高价
            self.market_data[f'{reqId}_high'] = price
        elif tickType == 7:  # 最低价
            self.market_data[f'{reqId}_low'] = price
        elif tickType == 9:  # 收盘价
            self.market_data[f'{reqId}_close'] = price
            
    def tickSize(self, reqId, tickType, size):
        """接收数量数据"""
        if tickType == 0:  # 买量
            self.market_data[f'{reqId}_bid_size'] = size
        elif tickType == 3:  # 卖量
            self.market_data[f'{reqId}_ask_size'] = size
        elif tickType == 5:  # 最新成交量
            self.market_data[f'{reqId}_last_size'] = size

# 创建合约对象的函数
def create_stock_contract(symbol, exchange='SMART', currency='USD'):
    """创建股票合约"""
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = exchange
    contract.currency = currency
    return contract

def create_index_contract(symbol, exchange='CBOE', currency='USD'):
    """创建指数合约"""
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'IND'
    contract.exchange = exchange
    contract.currency = currency
    return contract

def create_forex_contract(symbol, exchange='IDEALPRO'):
    """创建外汇合约"""
    contract = Contract()
    contract.symbol = symbol[:3]  # 基础货币
    contract.secType = 'CASH'
    contract.exchange = exchange
    contract.currency = symbol[3:]  # 计价货币
    return contract

def run_loop():
    app.run()

# 主程序
app = MarketDataApp()
app.connect('127.0.0.1', 7496, 123)

# 启动API线程
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# 等待连接
while not hasattr(app, 'nextorderId'):
    time.sleep(0.1)

print('开始请求市场数据...')

# 请求不同类型的数据
contracts = {
    1: create_stock_contract('AAPL'),      # 苹果股票
    2: create_stock_contract('TSLA'),      # 特斯拉股票
    3: create_index_contract('SPX'),       # S&P 500指数
    4: create_forex_contract('EURUSD'),    # 欧元美元
}

# 发送市场数据请求
for req_id, contract in contracts.items():
    app.reqMktData(req_id, contract, '', False, False, [])
    print(f'请求ID {req_id}: {contract.symbol}')

# 持续显示数据
try:
    for i in range(60):  # 运行60秒
        time.sleep(1)
        print(f'\n=== 第{i+1}秒数据 ===')
        for req_id in contracts:
            symbol = contracts[req_id].symbol
            bid = app.market_data.get(f'{req_id}_bid', 'N/A')
            ask = app.market_data.get(f'{req_id}_ask', 'N/A')
            last = app.market_data.get(f'{req_id}_last', 'N/A')
            print(f'{symbol}: 买价={bid}, 卖价={ask}, 最新价={last}')
            
except KeyboardInterrupt:
    print('用户中断程序')

# 取消数据订阅
for req_id in contracts:
    app.cancelMktData(req_id)

app.disconnect()
print('已断开连接')


ERROR -1 2104 市场数据场连接正常:usfarm.nj
ERROR -1 2104 市场数据场连接正常:hfarm
ERROR -1 2104 市场数据场连接正常:usfuture
ERROR -1 2104 市场数据场连接正常:cashfarm
ERROR -1 2104 市场数据场连接正常:usopt
ERROR -1 2104 市场数据场连接正常:eufarmnj
ERROR -1 2104 市场数据场连接正常:usfarm
ERROR -1 2106 历史市场数据场连接正常:euhmds
ERROR -1 2106 历史市场数据场连接正常:apachmds
ERROR -1 2106 历史市场数据场连接正常:fundfarm
ERROR -1 2106 历史市场数据场连接正常:ushmds
ERROR -1 2158 Sec-def数据场连接正常:secdefsg


连接成功
开始请求市场数据...
请求ID 1: AAPL
请求ID 2: TSLA
请求ID 3: SPX
请求ID 4: EUR


ERROR 4 10285 您的API版本不支持小数数量规则。请至少更新至版本163。



=== 第1秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334.49
SPX: 买价=-1.0, 卖价=-1.0, 最新价=N/A
EUR: 买价=N/A, 卖价=N/A, 最新价=N/A

=== 第2秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334.49
SPX: 买价=-1.0, 卖价=-1.0, 最新价=N/A
EUR: 买价=N/A, 卖价=N/A, 最新价=N/A

=== 第3秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334.49
SPX: 买价=-1.0, 卖价=-1.0, 最新价=N/A
EUR: 买价=N/A, 卖价=N/A, 最新价=N/A

=== 第4秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334.49
SPX: 买价=-1.0, 卖价=-1.0, 最新价=N/A
EUR: 买价=N/A, 卖价=N/A, 最新价=N/A

=== 第5秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334.49
SPX: 买价=-1.0, 卖价=-1.0, 最新价=N/A
EUR: 买价=N/A, 卖价=N/A, 最新价=N/A

=== 第6秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334.49
SPX: 买价=-1.0, 卖价=-1.0, 最新价=N/A
EUR: 买价=N/A, 卖价=N/A, 最新价=N/A

=== 第7秒数据 ===
AAPL: 买价=230.3, 卖价=230.44, 最新价=230.34
TSLA: 买价=334.36, 卖价=334.54, 最新价=334

In [17]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import pandas as pd

class HistoricalDataApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.historical_data = {}
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        
    def historicalData(self, reqId, bar):
        """接收历史数据"""
        if reqId not in self.historical_data:
            self.historical_data[reqId] = []
        
        self.historical_data[reqId].append({
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume
        })
        
    def historicalDataEnd(self, reqId, start, end):
        """历史数据接收完成"""
        print(f'请求ID {reqId} 的历史数据接收完成')
        df = pd.DataFrame(self.historical_data[reqId])
        print(f'共接收到 {len(df)} 条数据')
        print(df.tail())  # 显示最后5条数据

def run_loop():
    app.run()

# 主程序
app = HistoricalDataApp()
app.connect('127.0.0.1', 7496, 125)

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

while not hasattr(app, 'nextorderId'):
    time.sleep(0.1)

# 请求历史数据
contract = create_stock_contract('SPX')

# 参数说明：
# reqId: 请求ID
# contract: 合约对象
# endDateTime: 结束时间（空字符串表示当前时间）
# durationStr: 数据时间跨度（'1 D'=1天，'1 W'=1周，'1 M'=1月，'1 Y'=1年）
# barSizeSetting: K线周期（'1 min', '5 mins', '1 hour', '1 day'等）
# whatToShow: 数据类型（'TRADES'=成交价，'MIDPOINT'=中间价，'BID'=买价等）
# useRTH: 是否只使用常规交易时间（True/False）
# formatDate: 日期格式（1=字符串，2=时间戳）

app.reqHistoricalData(
    reqId=1,
    contract=contract,
    endDateTime='',
    durationStr='30 D',        # 获取30天数据
    barSizeSetting='1 day',    # 日K线
    whatToShow='TRADES',       # 成交价数据
    useRTH=True,              # 只使用常规交易时间
    formatDate=1,             # 字符串格式日期
    keepUpToDate=False,       # 不保持更新
    chartOptions=[]
)

# 等待数据
time.sleep(10)
app.disconnect()


ERROR -1 2104 市场数据场连接正常:usfarm.nj
ERROR -1 2104 市场数据场连接正常:hfarm
ERROR -1 2104 市场数据场连接正常:usfuture
ERROR -1 2104 市场数据场连接正常:cashfarm
ERROR -1 2104 市场数据场连接正常:usopt
ERROR -1 2104 市场数据场连接正常:eufarmnj
ERROR -1 2104 市场数据场连接正常:usfarm
ERROR -1 2106 历史市场数据场连接正常:euhmds
ERROR -1 2106 历史市场数据场连接正常:apachmds
ERROR -1 2106 历史市场数据场连接正常:fundfarm
ERROR -1 2106 历史市场数据场连接正常:ushmds
ERROR -1 2158 Sec-def数据场连接正常:secdefsg
ERROR 1 200 未找到该请求的证券定义


KeyboardInterrupt: 

In [18]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import pandas as pd

class HistoricalDataApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.historical_data = {}
        self.data_end_flags = {}
        
    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('连接成功，开始请求历史数据...')
        
    def error(self, reqId, errorCode, errorString):
        # 过滤正常状态消息
        if errorCode in [2104, 2106, 2158]:
            return  # 忽略正常状态通知
        else:
            print(f'错误 {errorCode}: {errorString}')
        
    def historicalData(self, reqId, bar):
        """接收历史数据"""
        if reqId not in self.historical_data:
            self.historical_data[reqId] = []
        
        self.historical_data[reqId].append({
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume
        })
        
    def historicalDataEnd(self, reqId, start, end):
        """历史数据接收完成"""
        symbol_names = {1: 'VIX', 2: 'SPX'}
        symbol_name = symbol_names.get(reqId, f'ID_{reqId}')
        
        print(f'\n=== {symbol_name} 历史数据接收完成 ===')
        
        if reqId in self.historical_data and self.historical_data[reqId]:
            df = pd.DataFrame(self.historical_data[reqId])
            print(f'共接收到 {len(df)} 条数据')
            print(f'数据时间范围: {df.iloc[0]["date"]} 到 {df.iloc[-1]["date"]}')
            print('最近5天数据:')
            print(df[['date', 'open', 'high', 'low', 'close']].tail())
            
            # 保存到CSV文件
            filename = f'{symbol_name}_historical_data.csv'
            df.to_csv(filename, index=False)
            print(f'数据已保存到 {filename}')
        else:
            print(f'{symbol_name}: 未接收到数据')
            
        self.data_end_flags[reqId] = True

def create_vix_contract():
    """创建VIX合约"""
    contract = Contract()
    contract.symbol = 'VIX'
    contract.secType = 'IND'  # 指数类型
    contract.exchange = 'CBOE'  # VIX在CBOE交易所
    contract.currency = 'USD'
    return contract

def create_spx_contract():
    """创建SPX合约"""
    contract = Contract()
    contract.symbol = 'SPX'
    contract.secType = 'IND'  # 指数类型
    contract.exchange = 'CBOE'  # SPX在CBOE
    contract.currency = 'USD'
    return contract

def run_loop():
    app.run()

# 主程序
app = HistoricalDataApp()
app.connect('127.0.0.1', 7496, 120)  # 7497为纸交易端口

# 启动API线程
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# 等待连接
while not hasattr(app, 'nextorderId'):
    time.sleep(0.1)

print('开始请求VIX和SPX历史数据...')

# 创建合约
vix_contract = create_vix_contract()
spx_contract = create_spx_contract()

# 请求VIX历史数据
app.reqHistoricalData(
    reqId=1,
    contract=vix_contract,
    endDateTime='',           # 空字符串表示当前时间
    durationStr='1 Y',        # 获取1年数据
    barSizeSetting='1 day',   # 日K线
    whatToShow='TRADES',      # 成交价数据
    useRTH=True,             # 只使用常规交易时间
    formatDate=1,            # 字符串格式日期
    keepUpToDate=False,      # 不保持更新
    chartOptions=[]
)

print('已发送VIX数据请求...')

# 等待VIX数据完成后再请求SPX（避免请求过于频繁）
time.sleep(2)

# 请求SPX历史数据
app.reqHistoricalData(
    reqId=2,
    contract=spx_contract,
    endDateTime='',
    durationStr='1 Y',        # 获取1年数据
    barSizeSetting='1 day',   # 日K线
    whatToShow='TRADES',
    useRTH=True,
    formatDate=1,
    keepUpToDate=False,
    chartOptions=[]
)

print('已发送SPX数据请求...')

# 等待数据接收完成
max_wait_time = 30  # 最长等待30秒
start_time = time.time()

while time.time() - start_time < max_wait_time:
    # 检查是否两个请求都完成了
    if len(app.data_end_flags) >= 2:
        print('\n所有数据请求已完成!')
        break
    time.sleep(1)

# 数据分析和比较
if 1 in app.historical_data and 2 in app.historical_data:
    vix_df = pd.DataFrame(app.historical_data[1])
    spx_df = pd.DataFrame(app.historical_data[1])
    
    print('\n=== 数据汇总 ===')
    print(f'VIX数据条数: {len(vix_df)}')
    print(f'SPX数据条数: {len(spx_df)}')
    
    # 最新数据
    if not vix_df.empty:
        latest_vix = vix_df.iloc[-1]
        print(f'\n最新VIX: {latest_vix["close"]:.2f} (日期: {latest_vix["date"]})')
    
    if not spx_df.empty:
        latest_spx = spx_df.iloc[-1]
        print(f'最新SPX: {latest_spx["close"]:.2f} (日期: {latest_spx["date"]})')
    
    # 合并数据保存
    if not vix_df.empty and not spx_df.empty:
        # 以日期为键合并数据
        vix_df['date'] = pd.to_datetime(vix_df['date'])
        spx_df['date'] = pd.to_datetime(spx_df['date'])
        
        merged_df = pd.merge(
            vix_df[['date', 'close']].rename(columns={'close': 'VIX_close'}),
            spx_df[['date', 'close']].rename(columns={'close': 'SPX_close'}),
            on='date', 
            how='inner'
        )
        
        merged_df.to_csv('VIX_SPX_combined.csv', index=False)
        print(f'\n合并数据已保存到 VIX_SPX_combined.csv (共{len(merged_df)}行)')

app.disconnect()
print('已断开连接')


连接成功，开始请求历史数据...
开始请求VIX和SPX历史数据...
已发送VIX数据请求...

=== VIX 历史数据接收完成 ===
共接收到 250 条数据
数据时间范围: 20240820 到 20250819
最近5天数据:
         date   open   high    low  close
245  20250813  14.62  14.81  14.30  14.49
246  20250814  14.66  15.39  14.57  14.83
247  20250815  14.43  15.18  14.40  15.09
248  20250818  15.73  15.95  14.95  14.99
249  20250819  15.23  15.23  15.05  15.05
数据已保存到 VIX_historical_data.csv
已发送SPX数据请求...

=== SPX 历史数据接收完成 ===
共接收到 250 条数据
数据时间范围: 20240819 到 20250818
最近5天数据:
         date     open     high      low    close
245  20250812  6395.17  6446.55  6385.76  6445.76
246  20250813  6462.67  6480.28  6445.02  6466.58
247  20250814  6453.46  6473.92  6441.07  6468.54
248  20250815  6477.38  6481.34  6441.85  6449.80
249  20250818  6445.02  6455.35  6437.70  6449.15
数据已保存到 SPX_historical_data.csv

所有数据请求已完成!

=== 数据汇总 ===
VIX数据条数: 250
SPX数据条数: 250

最新VIX: 15.05 (日期: 20250819)
最新SPX: 15.05 (日期: 20250819)

合并数据已保存到 VIX_SPX_combined.csv (共250行)
已断开连接
